Section 1: Import and Setup

In [19]:
import sys
import os
import numpy as np
from pathlib import Path
from datetime import datetime
import importlib  # For reloading modules during testing

# Add parent directory to path for imports
notebook_dir = Path.cwd()
project_root = notebook_dir.parent
sys.path.append(str(project_root))

# First import the modules
import aicons.definitions.simple_bad_aicon
import aicons.bayesbrainGPT.sensors.meta_s.meta_ads_sales_sensor

# Now reload them for development/testing
importlib.reload(aicons.definitions.simple_bad_aicon)
importlib.reload(aicons.bayesbrainGPT.sensors.meta_s.meta_ads_sales_sensor)
importlib.reload(aicons.bayesbrainGPT.bayes_brain)
importlib.reload(aicons.bayesbrainGPT.utility_function.marketing_utility)
importlib.reload(aicons.bayesbrainGPT.utility_function.utility_base)

# Import the classes after reloading
from aicons.definitions.simple_bad_aicon import SimpleBadAIcon
from aicons.bayesbrainGPT.sensors.meta_s.meta_ads_sales_sensor import MetaAdsSalesSensor

# Create the AIcon
aicon = SimpleBadAIcon("budget_allocation_aicon")
print(f"AIcon created: {aicon.name}")

AIcon created: budget_allocation_aicon


In [20]:
# Your Meta API credentials
access_token = "EAAZAn8wmq1IEBOZCz8oyDZBBgiazAgnQKIoAr4mFTbkV7jxi6t3APzOSxFybXNIkBgwQACdagbs5lFE8tpnNOBOOpWtS3KjZAdf9MNAlySpwEaDrX32oQwUTNmOZAaSXjT5Os5Q8YqRo57tXOUukB7QtcO8nQ8JuqrnnshCr7A0giynZBnJKfuPakrZBWoZD"  
ad_account_id = "act_252267674525035"     
campaign_id = "120218631288730217"           

# Create and add the Meta Ads Sensor
sensor = MetaAdsSalesSensor(
    name="meta_ads",
    reliability=0.9,
    access_token=access_token,
    ad_account_id=ad_account_id,
    campaign_id=campaign_id,
    api_version="v18.0",
    time_granularity="hour"
)

aicon.add_sensor("meta_ads", sensor)
print("Meta Ads sensor added")

# Get active ads to understand what we're working with
active_ads = sensor.get_active_ads()
print(f"Found {len(active_ads)} active ads")
for ad in active_ads:
    print(f"- Ad ID: {ad['ad_id']}, Name: {ad['ad_name']}")

# Check what factors were created
state_factors = aicon.brain.get_state_factors()
print(f"Number of factors: {len(state_factors)}")
print("Factor names:", ", ".join(state_factors.keys()))

✅ Successfully initialized Meta Ads API vv18.0 for account act_252267674525035
Found 4 ad sets in campaign 120218631288730217
Found 15 ads in 4 ad sets
Found 3 active ads out of 15 total ads
Found 3 active ads
- Ad ID: 120218907658700217, Name: Anuncio 4 - Carrusel 1 - Rancho
- Ad ID: 120219379327910217, Name: Anuncio 02 - Marzo - Foto - GABITO
- Ad ID: 120218905466570217, Name: Anuncio 2 - Carrusel 1 (3 artistas)
Number of factors: 21
Factor names: purchases, add_to_carts, initiated_checkouts, ad_120218907658700217_purchases, ad_120218907658700217_add_to_carts, ad_120218907658700217_initiated_checkouts, ad_120218907658700217_impressions, ad_120218907658700217_clicks, ad_120218907658700217_spend, ad_120219379327910217_purchases...
Auto-creating missing factor: purchases (continuous)
Creating continuous factor with TensorFlow: purchases
Added continuous factor with TensorFlow distribution: purchases
Auto-creating missing factor: add_to_carts (continuous)
Creating continuous factor with 

Create Action Space for Budget Allocation

In [21]:
# Create budget allocation action space for the active ad IDs
ad_ids = [ad['ad_id'] for ad in active_ads]
total_budget = 1000  # Set your total budget amount

print(f"Creating action space for {len(ad_ids)} ads with total budget ${total_budget}")
action_space = aicon.create_action_space(
    space_type='budget_allocation',
    total_budget=total_budget,
    items=ad_ids,
    step_size=0.01,  # 1% step size
    ignore_validation=True
)

# Create a utility function
utility_function = aicon.create_utility_function(
    utility_type='marketing_roi',
    revenue_per_sale=50.0  # $50 revenue per conversion
)

# Update beliefs with latest ad performance data
print("Updating beliefs from Meta Ads sensor...")
aicon.update_from_sensor("meta_ads")

# Now find the best action using the updated function
print("Finding optimal budget allocation...")
best_action, expected_profit = aicon.find_best_action(num_samples=500)

print("\nOPTIMAL BUDGET ALLOCATION:")
for ad_id, budget in best_action.items():
    # Find the ad name for this ID
    ad_name = next((ad['ad_name'] for ad in active_ads if ad['ad_id'] == ad_id), ad_id)
    print(f"- {ad_name}: ${budget:.2f}")

print(f"\nExpected profit: ${expected_profit:.2f}")

# You can also try with gradient optimization if you have TensorFlow utility functions
# best_action, expected_profit = aicon.find_best_action(num_samples=500, use_gradient=True)

Creating action space for 3 ads with total budget $1000
Created budget_allocation action space 'budget_allocation_space' with 3 dimensions
Created marketing_roi utility function: Marketing ROI Utility
Description: Calculates expected profit from ad spend
Updating beliefs from Meta Ads sensor...
Campaign objective: OUTCOME_SALES
Found 4 ad sets in campaign 120218631288730217
Found 15 ads in 4 ad sets
Trying date_preset: today
Fetching data for 15 ads with today preset...
Retrieved 69 records
Found latest hour: 20:00:00 - 20:59:59
Filtered to 2 records for hour 20:00:00 - 20:59:59
Found 2 hourly records
Extracted factors for 2 ads
Example ad IDs: 120218907658700217, 120219379327910217
Example factor names: purchases, add_to_carts, initiated_checkouts, ad_120218907658700217_purchases, ad_120218907658700217_add_to_carts, ad_120218907658700217_initiated_checkouts, ad_120218907658700217_impressions, ad_120218907658700217_clicks, ad_120218907658700217_spend, ad_120219379327910217_purchases
Ma

AttributeError: 'ActionSpace' object has no attribute 'generate_candidates'

Create utility function